In [2]:
import geopandas as gp
import pandas as pd
import os
import numpy as np
import pdv_functions as pdv
import re
from collections import Counter

# Alabama Election Results

## Load in and Clean Election Results

In [3]:
files = os.listdir('./raw-from-source/SOS/2022 General Precinct Level Results/')

county_list = []
for file in files: 
    # Load the file
    temp = pd.read_excel('./raw-from-source/SOS/2022 General Precinct Level Results/' + file)
    
    # Get the county name, clean "StClair" to match pattern
    county_name = file.split("-")[-1][0:-4]
    if county_name == "StClair":
        county_name = "St. Clair"
        
    # Clean the party name
    temp["Party"] = temp["Party"].str.strip()
    temp["Party"] = temp["Party"].fillna("")
    
    # Create a column to pivot on
    temp["pivot_col"] = temp["Contest Title"].str.strip()+"-:-"+temp["Candidate"].str.strip()
    temp["pivot_col"] = np.where(temp["Party"]=="",temp["pivot_col"],temp["pivot_col"]+"-:-"+temp["Party"].str.strip())
    
    # One-off cleaning of a candidate name
    temp["pivot_col"] = temp["pivot_col"].map({"STATE SENATOR, DISTRICT 6-:-Kyle Richard - Garrison-:-LIB":"STATE SENATOR, DISTRICT 6-:-Kyle Richard-Garrison-:-LIB"}).fillna(temp["pivot_col"])
    
    # Drop columns that are no longer needed
    temp.drop(["Contest Title", "Party", "Candidate"], axis = 1, inplace = True)
    
    # Add the county name to the precinct
    rename_dict = {i:i+"-:-"+county_name for i in temp.columns if i != "pivot_col"}
    temp.rename(columns = rename_dict, inplace = True)
    
    # Transpose the dataframe
    temp_transpose = temp.set_index("pivot_col").T
    temp_transpose.reset_index(inplace = True, drop = False)
    temp_transpose["County"] = county_name

    # Add to the list of counties
    county_list.append(temp_transpose)

# Concatenate into one file
comb = pd.concat(county_list)

## Filter Down Races

In [4]:
# Filter down to the races we want
comb_list = [i for i in comb.columns if "REPRESENTATIVE" in i] + [i for i in comb.columns if "GOVERNOR" in i] + [i for i in comb.columns if "SENATOR" in i] + [i for i in comb.columns if "ATTORNEY GENERAL" in i] + [i for i in comb.columns if "SECRETARY OF STATE" in i] + [i for i in comb.columns if "TREASURER" in i] + [i for i in comb.columns if "AUDITOR" in i] + [i for i in comb.columns if "AGRICULTURE" in i] + [i for i in comb.columns if "PUBLIC SERVICE" in i] + [i for i in comb.columns if "ASSOCIATE JUSTICE OF THE SUPREME COURT" in i] + [i for i in comb.columns if "CONSTITUTION OF ALABAMA" in i] + [i for i in comb.columns if "PROPOSED STATEWIDE AMENDMENT" in i]

# Remove the under votes and the over votes
comb_list = [i for i in comb_list if "Under Votes" not in i and "Over Votes" not in i]

## Rename Columns

In [5]:
def get_race(race_string):
    race_string = race_string.title()
    race_string = race_string.replace("(Vote For 1)","")
    if "U.S. House" in race_string or 'Us House' in race_string or "United States Representative" in race_string:
        return "CON"
    elif "United States Senator" in race_string:
        return "USS"
    elif "State House" in race_string or "State Representative" in race_string:
        return "SL"
    elif "State Senator" in race_string:
        return "SU"
    elif "President" in race_string:
        return "PRE"
    elif "US Senate" in race_string or "Us Senate" in race_string:
        return "USS"
    elif "Public Service" in race_string:
        if "1" in race_string:
            return "PS1"
        elif "2" in race_string:
            return "PS2"
    elif "Attorney General" in race_string:
        return "ATG"
    elif "Auditor General" in race_string or "State Auditor" in race_string:
        return "AUD"
    elif "Treasurer" in race_string:
        return "TRE"
    elif "Superintendent" in race_string:
        return "SUP"
    elif "Secretary Of State" in race_string:
        return "SOS"
    elif "Lieutenant Governor" in race_string:
        return "LTG"
    elif "Governor" in race_string:
        return "GOV"
    elif "Commissioner Of Labor" in race_string:
        return "LAB"
    elif "Commissioner Of Agriculture" in race_string:
        return "AGR"
    elif "Commissioner Of Insurance" in race_string:
        return "INS"
    elif "Associate Justice Of The Supreme Court" in race_string:
        if "5" in race_string:
            return "AJ5"
        elif "6" in race_string:
            return "AJ6"
    elif "Constitution" in race_string:
        return "CNS"
    elif "Amendment" in race_string:
        if "One " in race_string:
            return "A01"
        elif "Two " in race_string:
            return "A02"
        elif "Three " in race_string:
            return "A03"
        elif "Four " in race_string:
            return "A04"
        elif "Five " in race_string:
            return "A05"
        elif "Six " in race_string:
            return "A06"
        elif "Seven " in race_string:
            return "A07"
        elif "Eight " in race_string:
            return "A08"
        elif "Nine " in race_string:
            return "A09"
        elif "Ten " in race_string:
            return "A10"
        else:
            print("No race for:", race_string)
            raise ValueError
    elif "Referendum" in race_string:
        if "1" in race_string or "A" in race_string:
            return "RFA"
        elif "2" in race_string or "B" in race_string:
            return "RFB"
        else:
            print("No race for:", race_string)
            raise ValueError
    else:
        print("No race for:", race_string)
        raise ValueError
        
def get_election_type(race_string):
    if "(runoff)" in race_string:
        return "R"
    else:
        return "G"
        
def get_party(race_string):
    if "REP" in race_string:
        return "R"
    elif "DEM" in race_string or "(Democrat)" in race_string:
        return "D"
    elif "LIB" in race_string or "(L)" in race_string:
        return "L"
    elif "NON" in race_string:
        return "O"
    elif "IND" in race_string:
        return "I"
    elif race_string[0:3]=="Yes":
        return "YES"
    elif race_string[0:2]=="No":
        return "NO"
    else:
        print("NO RACE", race_string)
        return ""
           
def get_name(name_string):
    if "Write-In" in name_string:
        return "WRI"
    if "AMENDMENT" not in name_string and "Referendum" not in name_string and "CONSTITUTION" not in name_string:
        #print(name_string)
        #name_string = name_string.split(" (")[0]
        name_string = name_string.replace("'","")
        likely_last = name_string.split(" ")[-1]
        proposed_last = likely_last[:3]
        if proposed_last in ['II', 'III', 'Jr', 'Jr.', 'Sr.', 'JR.', "JR", "IV"]:
            likely_last = name_string.split(" ")[-2]
            proposed_last = likely_last[:3]
        #print(proposed_last.upper())
        return proposed_last.upper()
    else:
        return name_string.split("-:-")[1].upper()
#     name_string = name_string.split("-:-")[1]
#     name_string = name_string.replace(" (I)","")
#     name_string = name_string.replace("'","")
#     likely_last = name_string.split(" ")[-1]
#     proposed_last = likely_last[:3]
#     if proposed_last in ['II', 'III', 'Jr', 'Jr.', 'Sr.', 'JR.', "JR", "IV"]:
#         likely_last = name_string.split(" ")[-2]
#         proposed_last = likely_last[:3]
#     return proposed_last.upper()

def get_district(race_string, fill_level):
    race_string = race_string.split("-:-")[0]
    race_string = race_string.replace(" (Vote For 1)","")
    if "UNITED STATES REPRESENTATIVE" in race_string:
        break_word = "REPRESENTATIVE, "
        temp = race_string.split(break_word)[1]
        temp = re.findall('\d*', temp)[0]
    elif "STATE REPRESENTATIVE" in race_string or "STATE SENATOR" in race_string:
        break_word = "DISTRICT "
        temp = race_string.split(break_word)[1]
    else:
        raise ValueError
    
    return temp.zfill(fill_level)

def column_rename_function(name_string):
    election_type = get_election_type(name_string)
    year = "22"
    party = get_party(name_string.split("-:-")[-1])
    race = get_race(name_string)
    district = ""
    if race in ["CON", "SU"]:
        district = get_district(name_string, 2)
        year = ""
    elif race in ["SL"]:
        district = get_district(name_string, 3)
        year = ""
    
    name = get_name(name_string)
    if "CONSTITUTION" in name_string or "STATEWIDE AMENDMENT" in name_string:

        new_col_name = election_type + year + race + district +  name
    else:
        new_col_name = election_type + year + race + district + party + name
#         print(election_type)
#         print(year)
#         print(race)
#         print(district)
#         print(name)
    if len(new_col_name) > 10:
        print(name_string)
        print(new_col_name)
    return new_col_name

# Make a dictionary that points to the new column names and checks for duplicates
race_columns = comb_list

race_updates_dict = {}
race_updates_reversed = {}
clean_dups = {}
new_names = []
for val in race_columns:
    new_name = column_rename_function(val)
    race_updates_dict[val] = new_name
    if new_name not in new_names:
        new_names.append(new_name)
        race_updates_reversed[new_name] = val
    else:
        print("Duplicate", new_name)
        print(race_updates_reversed[new_name])
        print(val)
        clean_dups[val] = race_updates_reversed[new_name]

In [6]:
print([i for i in race_updates_dict.values() if i[0:3]=="G22"])
print([i for i in race_updates_dict.values() if i[0:3]!="G22"])

['G22GOVDFLO', 'G22GOVRIVE', 'G22GOVLBLA', 'G22GOVOWRI', 'G22LTGRAIN', 'G22LTGLPAG', 'G22LTGOWRI', 'G22USSDBOY', 'G22USSRBRI', 'G22USSLSOP', 'G22USSOWRI', 'G22ATGDMAJ', 'G22ATGRMAR', 'G22ATGOWRI', 'G22SOSDLAF', 'G22SOSRALL', 'G22SOSLSHE', 'G22SOSOWRI', 'G22TRERBOO', 'G22TRELHAM', 'G22TREOWRI', 'G22AUDRSOR', 'G22AUDLLAC', 'G22AUDOWRI', 'G22AGRRPAT', 'G22AGRLCLA', 'G22AGROWRI', 'G22PS1RODE', 'G22PS1LBIS', 'G22PS1OWRI', 'G22PS2RBEE', 'G22PS2LLAN', 'G22PS2OWRI', 'G22AJ5DKEL', 'G22AJ5RCOO', 'G22AJ5OWRI', 'G22AJ6RWIS', 'G22AJ6OWRI', 'G22CNSYES', 'G22CNSNO', 'G22A01YES', 'G22A01NO', 'G22A02YES', 'G22A02NO', 'G22A03YES', 'G22A03NO', 'G22A04YES', 'G22A04NO', 'G22A05YES', 'G22A05NO', 'G22A06YES', 'G22A06NO', 'G22A07YES', 'G22A07NO', 'G22A08YES', 'G22A08NO', 'G22A09YES', 'G22A09NO', 'G22A10YES', 'G22A10NO']
['GCON05DWAR', 'GCON05RSTR', 'GCON05LGRE', 'GCON05OWRI', 'GSL002RHAR', 'GSL002OWRI', 'GSL004RMOO', 'GSL004OWRI', 'GSL005RCRA', 'GSL005OWRI', 'GSL006RWHI', 'GSL006LTUR', 'GSL006OWRI', 'GSL025DH

## Sort the Column Names Alphabetically, Export for README

In [7]:
myKeys = list(race_updates_dict.values())
myKeys.sort()

sorted_dict = dict(sorted(race_updates_dict.items(), key=lambda x:x[1]))

export_dict = {i:key for key, i in sorted_dict.items()}

pd.DataFrame(export_dict.items()).to_csv("./field_names.csv", index = False)

## Rename the Columns

In [78]:
# Rename the columns
comb.rename(columns = sorted_dict, inplace = True)

# Filter down to needed columns
comb = comb[["index"]+list(sorted_dict.values())]

# Fillna
comb = comb.fillna(0)

# Define a county column
comb["County"] = comb["index"].apply(lambda x: x.split("-:-")[1])


# Allocate Absentee Votes

Note: An exception needs to be made for Jefferson County, as the provisional votes are reported countywide by the absentee votes are broken down into two different categories: "Birmingham" and "Bessemer" precincts

Note: I found the source data for this Jefferson information on their website

In [22]:
absentee_precs = comb[(comb["index"].str.contains("ABSENTEE"))|(comb["index"].str.contains("PROVISIONAL"))|(comb["index"].str.contains("PROVSIONAL"))]
nonabsentee_precs = comb[~((comb["index"].str.contains("ABSENTEE"))|(comb["index"].str.contains("PROVISIONAL"))|(comb["index"].str.contains("PROVSIONAL")))]

print(absentee_precs.shape)
print(nonabsentee_precs.shape)

(2074, 439)
(135, 440)
(1939, 440)


### Perform the Jefferson County Absentee Allocation
Note: This is where these absentee votes should go

In [23]:
bessemer_precs = ['PREC 1050 - MIDFIELD COMMUNITY-:-Jefferson',

       'PREC 1080 - MILES COLLEGE GYM-:-Jefferson',

       'PREC 1210 - GLEN OAKS ELEMENTA-:-Jefferson',

       'PREC 1240 - MT_ PILGRIM BAPTIS-:-Jefferson',

       'PREC 1285 - MINOR FIRE STATION-:-Jefferson',

       'PREC 1350 - FAIRFIELD CIVIC CE-:-Jefferson',

       'PREC 1365 - DOLOMITE W FIELD C-:-Jefferson',

       "PREC 1380 - ST_ MARY'S CATHOLI-:-Jefferson",

       'PREC 2060 - CHARLES A BROWN EL-:-Jefferson',

       'PREC 2110 - SOUTHSIDE HOMES CO-:-Jefferson',
       'PREC 2120 - DUNBAR-ABRAMS COMM-:-Jefferson',
       'PREC 2140 - NEW BEGINNING CHUR-:-Jefferson',
       'PREC 2150 - BRIGHTON COMMUNITY-:-Jefferson',

       'PREC 2210 - JONESBORO ELEMENTA-:-Jefferson',
       'PREC 2215 - GRANT STREET BAPTI-:-Jefferson',

       'PREC 2240 - FIRE DEPARTMENT AD-:-Jefferson',
       'PREC 2245/3060 - BESSEMER CIVI-:-Jefferson',

       'PREC 2340 - THOMPSON MANOR COM-:-Jefferson',
       'PREC 2350 - OXMOOR VALLEY COMM-:-Jefferson',
       'PREC 2360 - BESSEMER RECREATIO-:-Jefferson',
       'PREC 2365/3160 - ROSS BRIDGE W-:-Jefferson',
       'PREC 2370 - NEW BETHEL BAPTIST-:-Jefferson',
       'PREC 2390 - ROOSEVELT CITY COM-:-Jefferson',
       'PREC 2400 - WENONAH HIGH SCHOO-:-Jefferson',
       'PREC 2410 - BRYANT CHAPEL AME-:-Jefferson',

       'PREC 3010 - HOOVER MET BASEBAL-:-Jefferson',
       'PREC 3015 - HOOVER MET SPORTS-:-Jefferson',
       'PREC 3020 - OASIS OF PRAISE CH-:-Jefferson',
       'PREC 3025 - BAPTIST CHURCH OF-:-Jefferson',
       'PREC 3030 - PLEASANT GROVE FIR-:-Jefferson',
       'PREC 3035 - BETHEL BAPTIST CHU-:-Jefferson',
       'PREC 3040 - PRINCE OF PEACE CA-:-Jefferson',
       'PREC 3045 - BIRMINGHAM COMMUNI-:-Jefferson',
       'PREC 3050 - PLEASANT RIDGE BAP-:-Jefferson',

       'PREC 3080 - THE STATION CHURCH-:-Jefferson',
       'PREC 3090 - BROOKLANE COMMUNIT-:-Jefferson',
       'PREC 3110 - OAKMONT CHAPEL PRE-:-Jefferson',
       'PREC 3130 - SHADES CREST BAPTI-:-Jefferson',
       'PREC 3140 - VALLEY CREEK BAPTI-:-Jefferson',
       'PREC 3150 - UNION HILL BAPTIST-:-Jefferson',
       'PREC 3170 - ALLIANCE COMMUNITY-:-Jefferson',

       'PREC 3220 - OAK GROVE FIRST BA-:-Jefferson',
       'PREC 3240 - SYLVAN SPRINGS FIR-:-Jefferson',
       'PREC 3250 - BESSEMER FIRE STAT-:-Jefferson',
       'PREC 3260 - WEST JEFFERSON REC-:-Jefferson',
       'PREC 3270 - MUSCODA COMMUNITY-:-Jefferson',

       'PREC 3290 - MULGA TOWN HALL-:-Jefferson',
       'PREC 3300 - JOHNS COMMUNITY CE-:-Jefferson',
       'PREC 3310 - PARKWOOD CHURCH OF-:-Jefferson',
       'PREC 3320 - MAYTOWN BAPTIST CH-:-Jefferson',
       'PREC 3330 - FIRST BAPT CHURCH-:-Jefferson',

       'PREC 5010 - HOOVER PARK & RECR-:-Jefferson',

       'PREC 5030 - SHADES MOUNTAIN CO-:-Jefferson',

       'PREC 5100 - BLUFF PARK UNITED-:-Jefferson']


In [24]:
# Filter out the absentee precincts related to Birmingham or Bessemer
jefferson_exceptions = absentee_precs[absentee_precs["index"].str.contains("BIRMINGHAM") | absentee_precs["index"].str.contains("BESSEMER")]

# Make a list of the remaining absentee precincts
remaining_absentee =  absentee_precs[~(absentee_precs["index"].str.contains("BIRMINGHAM") | absentee_precs["index"].str.contains("BESSEMER"))]

In [25]:
# Going to use the names "BESSEMER" and "BIRMINGAHM" as the keys to allocate on and not the county names
jefferson_exceptions["Spec_Alloc"] = jefferson_exceptions["index"].apply(lambda x: x.split( )[0])

birmingham_precincts = nonabsentee_precs[(~nonabsentee_precs["index"].isin(bessemer_precs)) & (nonabsentee_precs["County"]=="Jefferson")]
bessemer_precincts = nonabsentee_precs[nonabsentee_precs["index"].isin(bessemer_precs)]

birmingham_precincts["Spec_Alloc"] = "BIRMINGHAM"
bessemer_precincts["Spec_Alloc"] = "BESSEMER"

In [29]:
receiving_jefferson = pd.concat([birmingham_precincts, bessemer_precincts])

In [30]:
# Get the precincts that need votes in the rest of the state
non_jefferson_non_absentee = nonabsentee_precs[~nonabsentee_precs["index"].isin(list(receiving_jefferson["index"]))]

In [31]:
jefferson_exceptions[["index", 'GSL016RSOU', 'GSL056DTIL']]

pivot_col,index,GSL016RSOU,GSL056DTIL
0,BESSEMER ABSENTEE-:-Jefferson,29.0,596.0
1,BIRMINGHAM ABSENTEE-:-Jefferson,3.0,22.0


In [32]:
# Zero Out These 25 Votes That Appear in the Wrong Place
jefferson_exceptions.loc[jefferson_exceptions["index"]=="BIRMINGHAM ABSENTEE-:-Jefferson","GSL016RSOU"] = 0
jefferson_exceptions.loc[jefferson_exceptions["index"]=="BIRMINGHAM ABSENTEE-:-Jefferson","GSL056DTIL"] = 0

In [33]:
tots = receiving_jefferson.groupby("Spec_Alloc").sum()
tots.reset_index(inplace = True, drop = False)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_80089/1156053499.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tots = receiving_jefferson.groupby("Spec_Alloc").sum()


In [34]:
tots[["Spec_Alloc", 'GSL016RSOU', 'GSL056DTIL']]

pivot_col,Spec_Alloc,GSL016RSOU,GSL056DTIL
0,BESSEMER,3765.0,8369.0
1,BIRMINGHAM,0.0,0.0


In [35]:
received_jefferson = pdv.allocate_absentee(receiving_jefferson, jefferson_exceptions, list(sorted_dict.values()), "Spec_Alloc", allocating_to_all_empty_precs=False)

/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:267: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_receiving_votes.loc[:,"Total_Votes"]+=df_receiving_votes.loc[:,race]
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:270: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  precinct_specific_totals = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:272: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a 

/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:313: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_receiving_votes.loc[:,floor_var]=0.0
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:311: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_receiving_votes.loc[:,add_var]=0.0
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:312: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:352: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  first_allocation = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:375: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_receiving_votes.loc[:,race] = df_receiving_votes.loc[:,race].astype(int)


In [36]:
received_jefferson.drop("Spec_Alloc", axis = 1, inplace = True)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_80089/810478150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  received_jefferson.drop("Spec_Alloc", axis = 1, inplace = True)


### Perform the Absentee Allocation for the Rest of the State

In [37]:
round_two = pd.concat([received_jefferson, non_jefferson_non_absentee])

In [40]:
round_two.reset_index(inplace = True, drop = True)
remaining_absentee.reset_index(inplace = True, drop = True)

In [43]:
final_allocation = pdv.allocate_absentee(round_two, remaining_absentee, list(sorted_dict.values()), "County", allocating_to_all_empty_precs=False)

/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:267: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_receiving_votes.loc[:,"Total_Votes"]+=df_receiving_votes.loc[:,race]
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:270: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  precinct_specific_totals = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:272: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a 

Special Allocation Needed [['Henry', 'G22SOSOWRI'], ['Henry', 'GCON02OWRI'], ['Perry', 'G22GOVOWRI']]


/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:311: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_receiving_votes.loc[:,add_var]=0.0
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:312: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_receiving_votes.loc[:,rem_var]=0.0
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:313: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has p

/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:311: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_receiving_votes.loc[:,add_var]=0.0
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:312: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_receiving_votes.loc[:,rem_var]=0.0
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:313: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has p

/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:352: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  first_allocation = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/pdv_functions.py:375: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_receiving_votes.loc[:,race] = df_receiving_votes.loc[:,race].astype(int)


## Final File Cleaning Before Export

In [45]:
county = pd.read_csv("/Users/peterhorton/Documents/RDH/raw_data/census/PL_COUNTYFP_NAMES.csv")
al_cnty_fips_dict = dict((zip(county[county["STUSAB"]=="AL"]["NAME"],county[county["STUSAB"]=="AL"]["COUNTYFP"].astype(str).str.zfill(3))))

In [46]:
final_allocation["Precinct"] = final_allocation["index"].apply(lambda x: x.split("-:-")[0])

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_80089/3327361160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_allocation["Precinct"] = final_allocation["index"].apply(lambda x: x.split("-:-")[0])


In [47]:
final_allocation["County_map"] = final_allocation["County"] + " County"

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_80089/2936216005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_allocation["County_map"] = final_allocation["County"] + " County"


In [48]:
final_allocation["COUNTYFP"] = final_allocation["County_map"].map(al_cnty_fips_dict).fillna(final_allocation["County"])

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_80089/3173388300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_allocation["COUNTYFP"] = final_allocation["County_map"].map(al_cnty_fips_dict).fillna(final_allocation["County"])


In [49]:
final_allocation["index"] = final_allocation["index"].apply(lambda x: x.split("-:-")[1]+"-:-"+x.split("-:-")[0])

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_80089/4103164098.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_allocation["index"] = final_allocation["index"].apply(lambda x: x.split("-:-")[1]+"-:-"+x.split("-:-")[0])


In [50]:
final_allocation.rename(columns = {"index":"UNIQUE_ID"}, inplace = True)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_80089/3659039950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_allocation.rename(columns = {"index":"UNIQUE_ID"}, inplace = True)


In [51]:
final_allocation = final_allocation[["UNIQUE_ID","COUNTYFP","County","Precinct"]+list(sorted_dict.values())]

In [55]:
stopping_point = final_allocation.copy(deep = True)

# Combining Election Result Precincts

Note: The votes in this precinct are split by last name and should be combined

In [56]:
print(final_allocation.loc[final_allocation["UNIQUE_ID"]=="Cullman-:-FAIRVIEW FIRE DEPT_ A-K"])

pivot_col                          UNIQUE_ID COUNTYFP   County  \
937        Cullman-:-FAIRVIEW FIRE DEPT_ A-K      043  Cullman   

pivot_col                 Precinct  G22A01NO  G22A01YES  G22A02NO  G22A02YES  \
937        FAIRVIEW FIRE DEPT_ A-K        60        278        78        237   

pivot_col  G22A03NO  G22A03YES  ...  GSU32OWRI  GSU32RELL  GSU33DFIG  \
937              60        260  ...          0          0          0   

pivot_col  GSU33OWRI  GSU33RRIE  GSU34OWRI  GSU34RWIL  GSU35LHUD  GSU35OWRI  \
937                0          0          0          0          0          0   

pivot_col  GSU35RSES  
937                0  

[1 rows x 442 columns]


In [57]:
final_allocation.loc[final_allocation["UNIQUE_ID"]=="Cullman-:-FAIRVIEW FIRE DEPT_ A-K", "UNIQUE_ID"] = "Cullman-:-FAIRVIEW FIRE DEPT TOWN HALL"

for race in list(sorted_dict.values()):
    final_allocation.loc[final_allocation["UNIQUE_ID"]=="Cullman-:-FAIRVIEW FIRE DEPT TOWN HALL", race] += final_allocation.loc[final_allocation["UNIQUE_ID"]=="Cullman-:-FAIRVIEW TOWN HALL_ L-Z", race].values[0] 

print(final_allocation.loc[final_allocation["UNIQUE_ID"]=="Cullman-:-FAIRVIEW FIRE DEPT TOWN HALL"])

final_allocation = final_allocation[final_allocation["UNIQUE_ID"]!="Cullman-:-FAIRVIEW TOWN HALL_ L-Z"]


pivot_col                               UNIQUE_ID COUNTYFP   County  \
937        Cullman-:-FAIRVIEW FIRE DEPT TOWN HALL      043  Cullman   

pivot_col                 Precinct  G22A01NO  G22A01YES  G22A02NO  G22A02YES  \
937        FAIRVIEW FIRE DEPT_ A-K       103        519       142        428   

pivot_col  G22A03NO  G22A03YES  ...  GSU32OWRI  GSU32RELL  GSU33DFIG  \
937             106        471  ...          0          0          0   

pivot_col  GSU33OWRI  GSU33RRIE  GSU34OWRI  GSU34RWIL  GSU35LHUD  GSU35OWRI  \
937                0          0          0          0          0          0   

pivot_col  GSU35RSES  
937                0  

[1 rows x 442 columns]


## Export Files

In [88]:
if not os.path.exists("./al_2022_gen_prec/"):
    os.mkdir("./al_2022_gen_prec/")

final_allocation.to_csv("./al_2022_gen_prec/al_2022_gen_prec.csv", index = False)

## Total Votes Checking

Note: All totals match those of the official PDF canvas except for the two contests where voters were zeroed manually

In [87]:
for col in final_allocation:
    if col[0]=="G":
        print(col.ljust(14), sum(final_allocation[col]))

G22A01NO       247554
G22A01YES      1020464
G22A02NO       256541
G22A02YES      939704
G22A03NO       219903
G22A03YES      994276
G22A04NO       235090
G22A04YES      937729
G22A05NO       355467
G22A05YES      780480
G22A06NO       437997
G22A06YES      676579
G22A07NO       272159
G22A07YES      834734
G22A08NO       272999
G22A08YES      686822
G22A09NO       273102
G22A09YES      683160
G22A10NO       278984
G22A10YES      816634
G22AGRLCLA     168529
G22AGROWRI     8265
G22AGRRPAT     946155
G22AJ5DKEL     454878
G22AJ5OWRI     1271
G22AJ5RCOO     943177
G22AJ6OWRI     25490
G22AJ6RWIS     998043
G22ATGDMAJ     450543
G22ATGOWRI     1220
G22ATGRMAR     955425
G22AUDLLAC     166781
G22AUDOWRI     8544
G22AUDRSOR     949871
G22CNSNO       273040
G22CNSYES      888456
G22GOVDFLO     412961
G22GOVLBLA     45958
G22GOVOWRI     9432
G22GOVRIVE     946932
G22LTGLPAG     178660
G22LTGOWRI     8103
G22LTGRAIN     957534
G22PS1LBIS     173287
G22PS1OWRI     8976
G22PS1RODE     937114
G22

~~G22A01NO       247554~~
~~G22A01YES      1020464~~
~~G22A02NO       256541~~
~~G22A02YES      939704~~
~~G22A03NO       219903~~
~~G22A03YES      994276~~
~~G22A04NO       235090~~
~~G22A04YES      937729~~
~~G22A05NO       355467~~
~~G22A05YES      780480~~
~~G22A06NO       437997~~
~~G22A06YES      676579~~
~~G22A07NO       272159~~
~~G22A07YES      834734~~
~~G22A08NO       272999~~
~~G22A08YES      686822~~
~~G22A09NO       273102~~
~~G22A09YES      683160~~
~~G22A10NO       278984~~
~~G22A10YES      816634~~
~~G22AGRLCLA     168529~~
~~G22AGROWRI     8265~~
~~G22AGRRPAT     946155~~
~~G22AJ5DKEL     454878~~
~~G22AJ5OWRI     1271~~
~~G22AJ5RCOO     943177~~
~~G22AJ6OWRI     25490~~
~~G22AJ6RWIS     998043~~
~~G22ATGDMAJ     450543~~
~~G22ATGOWRI     1220~~
~~G22ATGRMAR     955425~~
~~G22AUDLLAC     166781~~
~~G22AUDOWRI     8544~~
~~G22AUDRSOR     949871~~
~~G22CNSNO       273040~~
~~G22CNSYES      888456~~
~~G22GOVDFLO     412961~~
~~G22GOVLBLA     45958~~
~~G22GOVOWRI     9432~~
~~G22GOVRIVE     946932~~
~~G22LTGLPAG     178660~~
~~G22LTGOWRI     8103~~
~~G22LTGRAIN     957534~~
~~G22PS1LBIS     173287~~
~~G22PS1OWRI     8976~~
~~G22PS1RODE     937114~~
~~G22PS2LLAN     179883~~
~~G22PS2OWRI     8624~~
~~G22PS2RBEE     931354~~
~~G22SOSDLAF     436862~~
~~G22SOSLSHE     41611~~
~~G22SOSOWRI     663~~
~~G22SOSRALL     925267~~
~~G22TRELHAM     175034~~
~~G22TREOWRI     8855~~
~~G22TRERBOO     946936~~
~~G22USSDBOY     436746~~
~~G22USSLSOP     32879~~
~~G22USSOWRI     2459~~
~~G22USSRBRI     942154~~
~~GCON01LREM     26369~~
~~GCON01OWRI     1189~~
~~GCON01RCAR     140592~~
~~GCON02DHAR     58014~~
~~GCON02LREA     3396~~
~~GCON02OWRI     91~~
~~GCON02RMOO     137460~~
~~GCON03DVEA     47859~~
~~GCON03IBEL     3831~~
~~GCON03LCAS     3034~~
~~GCON03OWRI     80~~
~~GCON03RROG     135602~~
~~GCON04DNEI     26694~~
~~GCON04LCOC     4303~~
~~GCON04OWRI     81~~
~~GCON04RADE     164655~~
~~GCON05DWAR     62740~~
~~GCON05LGRE     6773~~
~~GCON05OWRI     369~~
~~GCON05RSTR     142435~~
~~GCON06LCHI     27833~~
~~GCON06OWRI     2137~~
~~GCON06RPAL     154233~~
~~GCON07DSEW     123233~~
~~GCON07LGOO     3212~~
~~GCON07OWRI     79~~
~~GCON07RNIC     67416~~
~~GSL001OWRI     319~~
~~GSL001RPET     10360~~
~~GSL002OWRI     2760~~
~~GSL002RHAR     11233~~
~~GSL003DTHO     4241~~
~~GSL003OWRI     23~~
~~GSL003RUND     8023~~
~~GSL004OWRI     334~~
~~GSL004RMOO     11236~~
~~GSL005OWRI     246~~
~~GSL005RCRA     12332~~
~~GSL006LTUR     2597~~
~~GSL006OWRI     147~~
~~GSL006RWHI     11052~~
~~GSL007DJON     3133~~
~~GSL007LDUR     304~~
~~GSL007OWRI     15~~
~~GSL007RYAR     11225~~
~~GSL008LWAL     1255~~
~~GSL008OWRI     46~~
~~GSL008RCOL     7234~~
~~GSL009LBOD     953~~
~~GSL009OWRI     29~~
~~GSL009RSTA     11041~~
~~GSL010DLAN     6608~~
~~GSL010LBOY     503~~
~~GSL010OWRI     4~~
~~GSL010RCOL     7581~~
~~GSL011OWRI     83~~
~~GSL011RSHE     12552~~
~~GSL012DFIE     2030~~
~~GSL012OWRI     23~~
~~GSL012RHAR     11958~~
~~GSL013LDAV     663~~
~~GSL013OWRI     29~~
~~GSL013RWOO     11422~~
~~GSL014OWRI     92~~
~~GSL014RWAD     13396~~
~~GSL015DROU     6166~~
~~GSL015OWRI     13~~
~~GSL015RHUL     10818~~
~~GSL016OWRI     98~~
~~GSL016RSOU     13174 (CEHCK)~~
~~GSL017OWRI     79~~
~~GSL017REST     12850~~
~~GSL018LSHI     571~~
~~GSL018OWRI     14~~
~~GSL018RKIE     10783~~
~~GSL019DHAL     8721~~
~~GSL019OWRI     322~~
~~GSL020OWRI     539~~
~~GSL020RLOM     14816~~
~~GSL021OWRI     523~~
~~GSL021RREY     12386~~
~~GSL022OWRI     254~~
~~GSL022RWHO     12984~~
~~GSL023OWRI     327~~
~~GSL023RKIR     10980~~
~~GSL024OWRI     87~~
~~GSL024RLED     11285~~
~~GSL025DHAG     7837~~
~~GSL025OWRI     33~~
~~GSL025RRIG     10813~~
~~GSL026DALF     1078~~
~~GSL026OWRI     10~~
~~GSL026RCOL     8258~~
~~GSL027DNEU     1583~~
~~GSL027OWRI     14~~
~~GSL027RKIT     13150~~
~~GSL028OWRI     290~~
~~GSL028RBUT     6566~~
~~GSL029LVAL     779~~
~~GSL029OWRI     35~~
~~GSL029RGID     12612~~
~~GSL030OWRI     92~~
~~GSL030RLIP     12325~~
~~GSL031OWRI     138~~
~~GSL031RSTU     11418~~
~~GSL032DBOY     5522~~
~~GSL032OWRI     9~~
~~GSL032RJAC     4390~~
~~GSL033DCRU     3571~~
~~GSL033OWRI     13~~
~~GSL033RROB     9534~~
~~GSL034OWRI     84~~
~~GSL034RSTA     12222~~
~~GSL035OWRI     148~~
~~GSL035RHUR     10880~~
~~GSL036OWRI     230~~
~~GSL036RWOO     12449~~
~~GSL037OWRI     154~~
~~GSL037RFIN     11937~~
~~GSL038LTEM     1161~~
~~GSL038OWRI     87~~
~~GSL038RWOO     9162~~
~~GSL039OWRI     114~~
~~GSL039RSHA     11224~~
~~GSL040DHOW     2221~~
~~GSL040OWRI     10~~
~~GSL040RROB     10772~~
~~GSL041DNEL     3203~~
~~GSL041LMOR     418~~
~~GSL041OWRI     14~~
~~GSL041RELL     10897~~
~~GSL042LWAR     931~~
~~GSL042OWRI     33~~
~~GSL042RSMI     12336~~
~~GSL043DCLE     3760~~
~~GSL043LBUR     644~~
~~GSL043OWRI     10~~
~~GSL043RMOO     12751~~
~~GSL044LBOO     2088~~
~~GSL044OWRI     108~~
~~GSL044RGAR     11396~~
~~GSL045LWHI     1950~~
~~GSL045OWRI     86~~
~~GSL045RDUB     12060~~
~~GSL046OWRI     418~~
~~GSL046RFAU     12656~~
~~GSL047DCOL     4816~~
~~GSL047OWRI     7~~
~~GSL047RSHA     8590~~
~~GSL048LSTU     2184~~
~~GSL048OWRI     84~~
~~GSL048RCAR     12518~~
~~GSL049OWRI     266~~
~~GSL049RBED     10604~~
~~GSL050OWRI     190~~
~~GSL050RHIL     12720~~
~~GSL051OWRI     198~~
~~GSL051RTRE     14025~~
~~GSL052DROG     10229~~
~~GSL052OWRI     89~~
~~GSL053DDAN     5748~~
~~GSL053OWRI     155~~
~~GSL054DRAF     9798~~
~~GSL054OWRI     111~~
~~GSL055DPLU     10018~~
~~GSL055OWRI     67~~
~~GSL056DTIL     8986 (CHECK)~~
~~GSL056LLES     869~~
~~GSL056OWRI     60~~
~~GSL057DSEL     9221~~
~~GSL057LJON     218~~
~~GSL057OWRI     21~~
~~GSL057RBAU     4752~~
~~GSL058DHOL     8385~~
~~GSL058OWRI     118~~
~~GSL059DMOO     8859~~
~~GSL059OWRI     72~~
~~GSL060DGIV     9872~~
~~GSL060LFRE     1126~~
~~GSL060OWRI     50~~
~~GSL061LPRU     1239~~
~~GSL061OWRI     64~~
~~GSL061RBOL     10546~~
~~GSL062DCEP     3094~~
~~GSL062OWRI     10~~
~~GSL062RLAM     8803~~
~~GSL063DADA     2827~~
~~GSL063OWRI     8~~
~~GSL063RALM     5541~~
~~GSL064LMAY     1133~~
~~GSL064OWRI     54~~
~~GSL064RGIV     11041~~
~~GSL065DCAS     4759~~
~~GSL065OWRI     4~~
~~GSL065REAS     12693~~
~~GSL066OWRI     114~~
~~GSL066RBAK     12042~~
~~GSL067DCHE     10335~~
~~GSL067OWRI     154~~
~~GSL068DJAC     9537~~
~~GSL068OWRI     10~~
~~GSL068RKEL     8981~~
~~GSL069DLAW     8769~~
~~GSL069OWRI     11~~
~~GSL069RMAD     7103~~
~~GSL070DENG     6245~~
~~GSL070OWRI     192~~
~~GSL071DMCC     9126~~
~~GSL071OWRI     199~~
~~GSL072DTRA     8128~~
~~GSL072OWRI     1785~~
~~GSL073OWRI     348~~
~~GSL073RPAS     10404~~
~~GSL074DENS     7538~~
~~GSL074OWRI     22~~
~~GSL074RMEA     4986~~
~~GSL075OWRI     324~~
~~GSL075RING     11690~~
~~GSL076DMCC     8312~~
~~GSL076LMAN     816~~
~~GSL076OWRI     40~~
~~GSL077DMOR     7062~~
~~GSL077OWRI     139~~
~~GSL078DHAS     6991~~
~~GSL078OWRI     160~~
~~GSL079LFRI     1692~~
~~GSL079OWRI     84~~
~~GSL079RLOV     7690~~
~~GSL080OWRI     202~~
~~GSL080RBLA     8113~~
~~GSL081OWRI     144~~
~~GSL081ROLI     12116~~
~~GSL082DWAR     6257~~
~~GSL082OWRI     22~~
~~GSL082RPIE     2526~~
~~GSL083DGRA     6540~~
~~GSL083OWRI     291~~
~~GSL084DFOR     7463~~
~~GSL084OWRI     277~~
~~GSL085DGRI     5668~~
~~GSL085OWRI     2~~
~~GSL085RREH     6665~~
~~GSL086OWRI     166~~
~~GSL086RLEE     10697~~
~~GSL087OWRI     98
~~GSL087RSOR     12957~~
~~GSL088LMAY     1524~~
~~GSL088OWRI     76~~
~~GSL088RSTA     10020~~
~~GSL089OWRI     155~~
~~GSL089RPAR     7855~~
~~GSL090OWRI     141~~
~~GSL090RSEL     11567~~
~~GSL091OWRI     146~~
~~GSL091RMAR     9794~~
~~GSL092DHUB     1795~~
~~GSL092OWRI     7~~
~~GSL092RHAM     11812~~
~~GSL093OWRI     192~~
~~GSL093RCLO     10333~~
~~GSL094LHEL     1815~~
~~GSL094OWRI     53~~
~~GSL094RFID     14207~~
~~GSL095DBRA     2465~~
~~GSL095OWRI     33~~
~~GSL095RHOL     14242~~
~~GSL096OWRI     284~~
~~GSL096RSIM     11990~~
~~GSL097DCLA     7337~~
~~GSL097OWRI     227~~
~~GSL098DBRA     7655~~
~~GSL098OWRI     332~~
~~GSL099DJON     8252~~
~~GSL099OWRI     349~~
~~GSL100LWAR     1627~~
~~GSL100OWRI     76~~
~~GSL100RSHI     9781~~
~~GSL101OWRI     195~~
~~GSL101RPRI     8968~~
~~GSL102OWRI     127~~
~~GSL102RSTR     10472~~
~~GSL103DDRU     6015~~
~~GSL103OWRI     219~~
~~GSL104LDEA     1960~~
~~GSL104OWRI     85~~
~~GSL104RWIL     8871~~
~~GSL105LLEW     1107~~
~~GSL105OWRI     45~~
~~GSL105RBRO     10554~~
~~GSU01OWRI      748~~
~~GSU01RMEL      35480~~
~~GSU02DLEW      20749~~
~~GSU02OWRI      43~~
~~GSU02RBUT      25951~~
~~GSU03LCHA      4566~~
~~GSU03OWRI      211~~
~~GSU03RORR      30578~~
~~GSU04OWRI      307~~
~~GSU04RGUD      38265~~
~~GSU05OWRI      296~~
~~GSU05RREE      36159~~
~~GSU06LRIC      4598~~
~~GSU06OWRI      129~~
~~GSU06RSTU      30883~~
~~GSU07DWIL      17295~~
~~GSU07OWRI      52~~
~~GSU07RGIV      29095~~
~~GSU08OWRI      614~~
~~GSU08RLIV      33758~~
~~GSU09OWRI      566~~
~~GSU09RSCO      32709~~
~~GSU10OWRI      598~~
~~GSU10RJON      32153~~
~~GSU11OWRI      543~~
~~GSU11RBEL      33505~~
~~GSU12DMCC      10271~~
~~GSU12OWRI      49~~
~~GSU12RKEL      28786~~
~~GSU13OWRI      597~~
~~GSU13RPRI      31759~~
~~GSU14OWRI      852~~
~~GSU14RWEA      31917~~
~~GSU15LCRU      5771~~
~~GSU15OWRI      260~~
~~GSU15RROB      35704~~
~~GSU16OWRI      1365~~
~~GSU16RWAG      33719~~
~~GSU17LFOR      3675~~
~~GSU17OWRI      122~~
~~GSU17RSHE      38137~~
~~GSU18DSMI      28642~~
~~GSU18OWRI      355~~
~~GSU19DCOL      26369~~
~~GSU19LWIL      3904~~
~~GSU19OWRI      204~~
~~GSU20DCOL      27567~~
~~GSU20OWRI      409~~
~~GSU21DWAR      11198~~
~~GSU21OWRI      132~~
~~GSU21RALL      21962~~
~~GSU22OWRI      404~~
~~GSU22RALB      32880~~
~~GSU23DSTE      25953~~
~~GSU23LSHE      959~~
~~GSU23OWRI      18~~
~~GSU23RNIM      21682~~
~~GSU24DSIN      25440~~
~~GSU24LBEN      5311~~
~~GSU24OWRI      259~~
~~GSU25LWOO      5394~~
~~GSU25OWRI      243~~
~~GSU25RBAR      32640~~
~~GSU26DHAT      25901~~
~~GSU26OWRI      403~~
~~GSU27DREE      10996~~
~~GSU27OWRI      63~~
~~GSU27RHOV      26014~~
~~GSU28DBEA      21252~~
~~GSU28LBOA      4837~~
~~GSU28OWRI      240~~
~~GSU29DMAT      6476~~
~~GSU29LMCB      737~~
~~GSU29OWRI      20~~
~~GSU29RCHE      31576~~
~~GSU30OWRI      602~~
~~GSU30RCHA      29225~~
~~GSU31OWRI      348~~
~~GSU31RCAR      30618~~
~~GSU32OWRI      768~~
~~GSU32RELL      41073~~
~~GSU33DFIG      23203~~
~~GSU33OWRI      51~~
~~GSU33RRIE      11401~~
~~GSU34OWRI      676~~
~~GSU34RWIL      27043~~
~~GSU35LHUD      4488~~
~~GSU35OWRI      189~~
~~GSU35RSES      27133~~